In [ ]:
import pandas as pd
import numpy as np
import urllib.request 
import os
import glob
import os.path
import csv
import re

In [ ]:
file_paths=[]
# after we loop through the folder and get every csv file, we go inside the csv file and grab links for each csv file
path_2022_03 = "/project/ll_774_951/uk_ru/twitter/data/2022-03/*.csv"
for fname in glob.glob(path_2022_03):
    file_paths.append(fname)
print(len(file_paths),'csv files in the 2022-03 folder')
#file_path

In [ ]:
#ath = "/project/ll_774_951/uk_ru/twitter/data/2022-03/ukraine_russia-2022-03-16-00.csv"
def combine_df(path,counter):
    # combine all files three columns (media_urls, rt_media_urls, q_media_urls) into one list
    # and store it to a csv file
    special_char = ",]['"
    image_links=pd.DataFrame(index=['urls'])
    for csv in path: # try 2 files
        df = pd.read_csv(csv,engine='python', error_bad_lines=False)
        df_urls = df[['media_urls','rt_media_urls','q_media_urls']]
        df_all_urls = df_urls.melt(value_name='urls')[['urls']] # into single column
        download_1 = df_all_urls.loc[df_all_urls.urls.str.contains('http', na=False)]
        download_1 = download_1.replace("\[\'",'',regex=True)
        download_1 = download_1.replace("\'\]",'',regex=True)
        download_1 = download_1.replace("\,",'',regex=True)
        # download_list_1 = download_1["urls"].tolist()
        # downloads_1 = [''.join(x for x in string if not x in special_char) for string in download_list_1]
        #downloads_set =set(downloads_1)
        #downloads_list =list(downloads_set)
        image_links.append(download_1, ignore_index=True)
    # flat_image_links = list(np.concatenate(image_links)) # it was a list of lists
    # flat_image_links_set = set(flat_image_links)
    # flat_image_links_list = list(flat_image_links_set)
    # ls_csv_one_column = flat_image_links_list.to_csv(str(counter)+'.csv')
    ## need to change the above line for storing the csv to the correct path on HPC
    image_links.to_csv(os.path.join('/project/ll_774_951/uk_ru/twitter/one_column_urls/',str(counter)+'.csv'))
    return image_links # listofurls

In [ ]:
def download_images(one_column_csv):
    df = pd.read_csv('/project/ll_774_951/uk_ru/twitter/one_column_urls/'+str(one_column_csv)+'.csv')
    try:
        for link in df:
            urllib.request.urlretrieve(link,'/project/ll_774_951/uk_ru/twitter/twitter_images2/'+os.path.basename(link))
    except:
        pass

In [ ]:
batch_size = 1
counter = 0 # define a counter so that we can keep track which csv file we have download already
for i in range(0,len(file_paths[0:1]),batch_size):
    counter+=1
    Batch = file_paths[i:i+batch_size]
    single_column_csv=combine_df(Batch,counter)
    download_images(single_column_csv)
    print('success'+str(counter))